In [ ]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
!pip install tqdm

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# # Load data
# train = pd.read_csv('/content/drive/MyDrive/MBZ/Thesis/bert with features/train_data_with_allfeats.csv')
# test = pd.read_csv('/content/drive/MyDrive/MBZ/Thesis/bert with features/test_data_with_allfeats.csv')
# dev = pd.read_csv('/content/drive/MyDrive/MBZ/Thesis/bert with features/dev_data_with_allfeatures_final.csv')
data_path = '/content/drive/MyDrive/MBZ/Thesis/bert with features/data_with_all_features_novocab.csv' # DATA PATH

all_df = pd.read_csv(data_path, header=0)
all_df = all_df[all_df['RL_num_19']>= 12]


In [ ]:

all_df_grouped = all_df.groupby('Split')

# all_df = all_df[[DATA_COLUMN, LABEL_COLUMN]]
# all_df.columns = [DATA_COLUMN, LABEL_COLUMN]

train_df = all_df_grouped.get_group('Train')
#train_df = train_df.head(4565)
dev_df = all_df_grouped.get_group('Dev')
test_df = all_df_grouped.get_group('Test')
#tune_df = all_df.get_group('Tune')

In [ ]:
import datasets
from datasets import load_dataset


In [ ]:
train = datasets.Dataset.from_pandas(train_df)
dev = datasets.Dataset.from_pandas(dev_df)
test = datasets.Dataset.from_pandas(test_df)
#tune = datasets.Dataset.from_pandas(tune_df)
dataset = load_dataset("labr") #dump loading .. only to match the dataset template from huggingface
dataset['train'] = train
#dataset['tune'] = tune
dataset['dev'] = dev
dataset['test'] = test

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.02k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.83M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/919k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11760 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2935 [00:00<?, ? examples/s]

In [ ]:
train_texts = dataset['train']['word_sents']
train_labels = dataset['train']['RL_num_19']

dev_texts = dataset['dev']['word_sents']
dev_labels = dataset['dev']['RL_num_19']

test_texts = dataset['test']['word_sents']
test_labels = dataset['test']['RL_num_19']
# Choose text and label
# texts = df["word_sents"].tolist()
# labels = df["RL_num_19"].astype(int).tolist()

# Drop columns not part of the feature set
non_feature_cols = ['ID', 'Sentence', 'Word Count','Readability Level', 'RL_num_19', 'Annotator', 'annotation file', 'Project Phase', 'Source', 'Book', 'Author', 'Domain', 'Text Class', 'Reader Class', 'File', 'Split',
       'Clean_Sentnece', 'word count unique', 'max word count in RL',
       'RL_num_7', 'RL_num_5', 'RL_num_3', 'd3lex_sents', 'lex_sents',
       'd3tok_sents', 'word_sents', 'syllable_word']
# relevant_columns =  ['jar_majroor', 'relative_pronoun_singular', 'negation_particle', 'verb_past_s_p', 'coordinated_verbs', 'verb_command', 'word count unique', 'broken_plural', 'verbal_sentence_with_two_objects', 'vocative', 'verbal_present_sentence_with_an_almasdariya', 'suf_pron', 'plural_masc', 'verb_present_plural', 'inna_wa_akhawataha', 'plural_fem_noun_adj', 'kana_wa_akhawataha', 'interrogative_alif', 'syllables', 'passive_voice', 'relative_pronoun_dual_plural', 'verb_past_present_dual', 'advanced_khabar', 'amma_lakin'] #[col for col in data.columns if col not in irrelevant_columns and col != 'RL_num_19']
relevant_columns = [col for col in all_df.columns if col not in non_feature_cols]


train_features = train_df[relevant_columns]
train_features = train_features.astype(float).values

dev_features = dev_df[relevant_columns]
dev_features = dev_features.astype(float).values

test_features = test_df[relevant_columns]
test_features = test_features.astype(float).values

# feature_df = all_df.drop(columns=non_feature_cols)
# features = feature_df.astype(float).values
feature_dim = train_features.shape[1]

train_labels = [label - 12 for label in train_labels]
dev_labels = [label - 12 for label in dev_labels]
test_labels = [label - 12 for label in test_labels]

In [ ]:
# subtract one from each train_labels


In [ ]:
feature_dim

In [ ]:
# Tokenization
tokenizer = BertTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02")
encodings = tokenizer(train_texts, padding=True, truncation=True, return_tensors="pt")

# Dataset Class
class BERTDatasetWithFeatures(Dataset):
    def __init__(self, encodings, features, labels):
        self.encodings = encodings
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'features': self.features[idx],
            'labels': self.labels[idx]
        }

tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/825k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

In [ ]:
# # Model
# class BERTWithFeatures(nn.Module):
#     def __init__(self, bert_model_name='aubmindlab/bert-base-arabertv02', feature_dim=51, num_labels=19):
#         super(BERTWithFeatures, self).__init__()
#         self.bert = BertModel.from_pretrained(bert_model_name)
#         self.dropout = nn.Dropout(0.3)
#         self.classifier = nn.Linear(self.bert.config.hidden_size + feature_dim, num_labels)

#     def forward(self, input_ids, attention_mask, features):
#         outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
#         cls_output = outputs.last_hidden_state[:, 0, :]
#         combined = torch.cat((cls_output, features), dim=1)
#         combined = self.dropout(combined)
#         return self.classifier(combined)

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel

class BERTWithFeatures(nn.Module):
    def __init__(self, bert_model_name='aubmindlab/bert-base-arabertv02', feature_dim=51, feature_embed_dim=128, num_labels=19):
        super(BERTWithFeatures, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.feature_dense = nn.Linear(feature_dim, feature_embed_dim)  # 🔧 Embedding layer for features
        self.dropout = nn.Dropout(0.3)

        combined_dim = self.bert.config.hidden_size + feature_embed_dim
        self.classifier = nn.Linear(combined_dim, num_labels)

    def forward(self, input_ids, attention_mask, features):
        # BERT CLS output
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]

        # Embed features
        feature_embedding = torch.relu(self.feature_dense(features))

        # Concatenate and classify
        combined = torch.cat((cls_output, feature_embedding), dim=1)
        combined = self.dropout(combined)
        return self.classifier(combined)





In [ ]:
train_encodings = tokenizer(train_texts, padding=True, truncation=True, return_tensors="pt")
val_encodings = tokenizer(dev_texts, padding=True, truncation=True, return_tensors="pt")

train_dataset = BERTDatasetWithFeatures(train_encodings, train_features, train_labels)
val_dataset = BERTDatasetWithFeatures(val_encodings, dev_features, dev_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

In [ ]:


# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTWithFeatures(feature_dim=feature_dim).to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()


model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

# Define metric functions
def macro_f1(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro')

def macro_precision(y_true, y_pred):
    return precision_score(y_true, y_pred, average='macro')

def macro_recall(y_true, y_pred):
    return recall_score(y_true, y_pred, average='macro')

def acc(y_true, y_pred):
    return accuracy_score(y_true, y_pred)



In [ ]:
from tqdm import tqdm

EPOCHS = 3

for epoch in range(EPOCHS):
    print(f"\n🌟 Epoch {epoch+1}/{EPOCHS}")
    model.train()
    total_loss = 0

    train_iter = tqdm(train_loader, desc="🔁 Training", leave=False)
    for batch in train_iter:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        features = batch['features'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask, features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        train_iter.set_postfix(loss=loss.item())

    avg_train_loss = total_loss / len(train_loader)
    print(f"  🔧 Average Training Loss: {avg_train_loss:.4f}")

    # Evaluation
    model.eval()
    val_preds, val_labels_all = [], []

    val_iter = tqdm(val_loader, desc="🔍 Evaluating", leave=False)
    with torch.no_grad():
        for batch in val_iter:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            features = batch['features'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask, features)
            _, preds = torch.max(outputs, dim=1)

            val_preds.extend(preds.cpu().numpy())
            val_labels_all.extend(labels.cpu().numpy())

    # Metrics
    f1 = macro_f1(val_labels_all, val_preds)
    precision = macro_precision(val_labels_all, val_preds)
    recall = macro_recall(val_labels_all, val_preds)
    accuracy = acc(val_labels_all, val_preds)

    print(f"  📊 Val Accuracy:        {accuracy:.4f}")
    print(f"  🎯 Macro F1 Score:      {f1:.4f}")
    print(f"  🧠 Macro Precision:     {precision:.4f}")
    print(f"  💡 Macro Recall:        {recall:.4f}")



🌟 Epoch 1/3


  🔧 Average Training Loss: 1.1116


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


  📊 Val Accuracy:        0.6310
  🎯 Macro F1 Score:      0.2782
  🧠 Macro Precision:     0.3889
  💡 Macro Recall:        0.2747

🌟 Epoch 2/3


  🔧 Average Training Loss: 0.8894


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


  📊 Val Accuracy:        0.6278
  🎯 Macro F1 Score:      0.3135
  🧠 Macro Precision:     0.3429
  💡 Macro Recall:        0.3077

🌟 Epoch 3/3


  🔧 Average Training Loss: 0.6902


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


  📊 Val Accuracy:        0.6181
  🎯 Macro F1 Score:      0.2818
  🧠 Macro Precision:     0.3069
  💡 Macro Recall:        0.2816


In [ ]:
test_encodings = tokenizer(test_texts, padding=True, truncation=True, return_tensors="pt")

test_dataset = BERTDatasetWithFeatures(test_encodings, test_features, test_labels)

test_loader = DataLoader(test_dataset, batch_size=16)


In [ ]:
model.eval()
all_preds = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="🚀 Predicting"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        features = batch['features'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, features=features)
        _, preds = torch.max(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())


🚀 Predicting: 100%|██████████| 249/249 [00:08<00:00, 30.54it/s]


In [ ]:
all_preds = [label + 12 for label in all_preds]


In [ ]:
test_df["Predicted_RL"] = all_preds
test_df.to_csv("/content/drive/MyDrive/MBZ/Thesis/bert with features/12levels_test_predictions_relu.csv", index=False)
print("✅ Predictions saved to test_predictions.csv")


<ipython-input-18-b8e4fe38b64f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["Predicted_RL"] = all_preds


✅ Predictions saved to test_predictions.csv


In [ ]:
acc(val_labels_all, val_preds)

0.6181434599156118

In [ ]:
dev_df['Predicted_RL'] = val_preds
dev_df.to_csv("/content/drive/MyDrive/MBZ/Thesis/bert with features/12levels_dev_predictions_relu.csv", index=False)
print("✅ Predictions saved to dev_predictions.csv")

<ipython-input-20-32494cf9319a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dev_df['Predicted_RL'] = val_preds


✅ Predictions saved to dev_predictions.csv
